# Load Data

In [1]:
import pandas as pd

### FOMC Rates

In [2]:
import os


In [3]:
os.listdir()

['page2.md',
 'pics',
 'README.md',
 'example.ipynb',
 'Load_Data.ipynb',
 '.gitignore',
 '_config.yml',
 '.ipynb_checkpoints',
 '.git',
 'notebooks',
 'inputs']

In [4]:
rates = pd.read_csv('inputs/FOMC_rates.csv')

In [5]:
rates

,Date,Increase,Decrease,Tgt_level
0,3/23/23,25,0,4.75-5.00
1,2/2/23,25,0,4.50-4.75
2,12/15/22,50,0,4.25-4.50
3,11/3/22,75,0,3.75-4.00
4,9/22/22,75,0,3.00-3.25
5,7/28/22,75,0,2.25-2.50
6,6/16/22,75,0,1.50-1.75
7,5/5/22,50,0,0.75-1.00
8,3/17/22,25,0,0.25-0.50
9,3/16/20,0,100,0-0.25


Source: https://www.federalreserve.gov/monetarypolicy/openmarket.htm

### Index Returns

In [9]:
sp500_ret = pd.read_csv('inputs/SP500_ret.csv')

In [10]:
sp500_ret.head(15)

,Effective date,S&P 500
0,1/2/15,2058.20
1,1/5/15,2020.58
2,1/6/15,2002.61
3,1/7/15,2025.90
4,1/8/15,2062.14
5,1/9/15,2044.81
6,1/12/15,2028.26
7,1/13/15,2023.03
8,1/14/15,2011.27
9,1/15/15,1992.67


### Long Term Bond Returns

### Short Term Bond Reutrns

### Stock Trading Volume

In [ ]:
stock_tv = pd.read_csv('inputs/SP500_TV.csv')

In [ ]:
stock_tv

Source: https://ycharts.com/indicators/cboe_spx_volume

### Long Term Bond Trading Volume

### Short Term Bond Trading Volume